In [4]:
import numpy as np
import pandas as pd

! pip install transformers
! pip install sacremoses

In [5]:
from typing import Callable, List, Optional, Tuple
import pandas as pd
from sklearn.base import TransformerMixin, BaseEstimator
import torch

class BertTransformer(BaseEstimator, TransformerMixin):
    def __init__(
            self,
            bert_tokenizer,
            bert_model,
            max_length: int = 60,
            embedding_func: Optional[Callable[[torch.tensor], torch.tensor]] = None,
    ):
        self.bert_tokenizer = bert_tokenizer
        self.bert_model = bert_model
        self.bert_model.eval()
        self.max_length = max_length
        self.embedding_func = embedding_func

        if self.embedding_func is None:
            self.embedding_func = lambda x: x[0][:, 0, :].squeeze()

    def _tokenize(self, text: str) -> Tuple[torch.tensor, torch.tensor]:
        tokenized_text = self.bert_tokenizer.encode_plus(text,
                                                         add_special_tokens=True,
                                                         max_length=self.max_length
                                                         )["input_ids"]

        attention_mask = [1] * len(tokenized_text)

        return (
            torch.tensor(tokenized_text).unsqueeze(0),
            torch.tensor(attention_mask).unsqueeze(0),
        )

    def _tokenize_and_predict(self, text: str) -> torch.tensor:
        tokenized, attention_mask = self._tokenize(text)

        embeddings = self.bert_model(tokenized, attention_mask)
        return self.embedding_func(embeddings)

    def transform(self, text: List[str]):
        if isinstance(text, pd.Series):
            text = text.tolist()

        with torch.no_grad():
            return torch.stack([self._tokenize_and_predict(string) for string in text])

    def fit(self, X, y=None):
        return self


c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from transformers import AutoTokenizer, AutoModel

model_names = {
    "herbert-klej-cased-v1": {
        "tokenizer": "allegro/herbert-klej-cased-tokenizer-v1",
        "model": "allegro/herbert-klej-cased-v1",
    },
    "herbert-base-cased": {
        "tokenizer": "allegro/herbert-base-cased",
        "model": "allegro/herbert-base-cased",
    },
    "herbert-large-cased": {
        "tokenizer": "allegro/herbert-large-cased",
        "model": "allegro/herbert-large-cased",
    },
}

tokenizer = AutoTokenizer.from_pretrained(model_names["herbert-base-cased"]["tokenizer"])
bert_model = AutoModel.from_pretrained(model_names["herbert-base-cased"]["model"]) 
bert_transformer = BertTransformer(tokenizer, bert_model)

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.sso.sso_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.sso.sso_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
file_name = 'GDT_data_v01.xlsx'
df = pd.read_excel(io=file_name)

df['sex'] = [(male, female) for male, female in zip(df['is_male'], df['is_female'])]
df['nlp_2_4_5'] = df['nlp_4'] + df['nlp_5'] + df['nlp_2']

In [15]:
gd_category = []

for item in df['GDT_score']:
    if item <= 8:
        category = 0
    elif item >= 12:
        category = 1

    gd_category.append(category)

df['GD_category'] = gd_category

In [27]:
data_1 = df[df['GD_category']==1]
data_2 = df[df['GD_category']==0].sample(n=110)

working_data = pd.concat([data_1, data_2])

,index,age,is_male,is_female,GAD_PHQ_score,GAD_PHQ_score_2,GDT_score,GD_label,harmony_score,nlp_1,nlp_2,nlp_3,nlp_4,nlp_5,nlp_6,sex,nlp_2_4_5,GD_category
3,5,36,1,0,34,34.0,14,0,35,"W pełnym stopniu. Bardzo dobrze zarabiam, stać...",Odpręża. Relaksuje. Odpręża. Zapewnia spokój. ...,NaN,NaN,NaN,NaN,"(1, 0)",NaN,1
6,8,41,1,0,46,46.0,18,1,22,"Jestem w stanie zrealizować w 100%, jestem bar...",Bardzo dobrze. Elegancko. Wspaniale. Bez probl...,Dobrze. Wspaniale. Rozwijam się. Szczęśliwy. D...,Wszystkie. Spokój. Zadbanie. Cisza. Uspokojenie.,Nie lubią. Źli. Krzyczą. Radują się. Grają.,Poprawia. Zaspokaja. Uspokaja. Daje bezpieczeń...,"(1, 0)",Wszystkie. Spokój. Zadbanie. Cisza. Uspokojeni...,1
10,12,35,1,0,24,48.0,12,0,30,Jestem dość dobrze w stanie realizować moje ce...,Relaksuje. Pozwala zabić czas. Jest przyjemne....,Nie wpływa,Zabicie czasu. Relaks. Zapomnienie. Wyciszenie...,Obojętnie. Zniecierpliwienie. Zaciekawienie. I...,Spokojny. Stabilny. Znudzony. Zbyt zajęty. Obo...,"(1, 0)",Zabicie czasu. Relaks. Zapomnienie. Wyciszenie...,1
13,15,22,1,0,39,78.0,13,0,23,jestem w stanie zrealizować swoje cele życiowe...,Super. Ładne. Czas. Fajnie. Rozrywka.,Nerwy. Frustracja. Wkuriwenie. Mega. Dół.,Czas. Ciekawosc. Relacje. Spedzonie czasu. Milo.,Nerwy. Strata. Czas. Opcja. Fajnie.,Nerwy. Słabe. Wyzwanie. Dobrze.,"(1, 0)",Czas. Ciekawosc. Relacje. Spedzonie czasu. Mil...,1
14,16,44,1,0,22,44.0,11,0,30,"Mam sporo celów przed sobą, niestety wychowani...",Zabiera czas. Odpręża. Kosztuje. Denerwuje. Za...,Nie myślę o tym. Mam czas dla innych. Śpię. Sp...,Odreagowania. Wyżycia się. Aktywności. Satysfa...,Nerwowo. Z ciekawością. Pytajaco. Coś w zamian...,Opanowana. Spokojna. Rozważna. Wesoła. Zrelaks...,"(1, 0)",Odreagowania. Wyżycia się. Aktywności. Satysfa...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,218,23,0,1,39,78.0,6,0,8,Nie jestem w stanie realizować swoich celli ży...,Nuda. Ciekawość. Myslenie. Zabawa. Strata czasu.,Wyobraźnia.,Zaspokaja nude. Zaspakaja moje myśli.,Grają ze mną. Są ciekawi. Rywalizujemy.,Bardzo dobra,"(0, 1)",Zaspokaja nude. Zaspakaja moje myśli.Grają ze ...,0
452,462,27,0,1,42,84.0,8,0,27,Jestem w stanie realizować swoje cele życiowe....,Negatywny. Czasochłonny. Męczący. Nudny. Absor...,Irytacja. Zlosc. Nuda. Zazdrość. Wstyd.,Rywalizacja. Zachwyt. Ambicje. Refleks. Radość.,Złość. Stres. Nerwy. Irytacja. Wściekłość.,Dobra. Zadowalająca. Stabilna. Silna. Bardzo d...,"(0, 1)",Rywalizacja. Zachwyt. Ambicje. Refleks. Radość...,0
504,520,18,0,1,31,31.0,5,0,27,"Tak, jestem w stanie realizować swoje cele życ...",Logiczne myślenie. Doskonalą refleks. Nauka ję...,Zdenerwowanie. Podekscytowanie. Zniecierpliwie...,Rozrywka. Zajmuje czas. Nauka.,Akceptacja. Niepokój. Złość. Zrozumienie. .,Dobra.,"(0, 1)",Rozrywka. Zajmuje czas. Nauka.Akceptacja. Niep...,0
350,360,20,0,1,47,141.0,8,0,10,"Niestety nie, mam problem z wyznaczeniem swoic...",Zajmuje czas. Bardzo pochłania. Czas ze znajom...,Zadowolenie. Szczęście. Satysfakcja. Czas woln...,Odstresowuje. Wyłącza myslenie. Odpoczynek. Re...,Wyrozumiale. Denerwują czasami. Rozumieją. Gra...,Zmeczona. Obojetna. Smutna. Zawiedziona. Nieko...,"(0, 1)",Odstresowuje. Wyłącza myslenie. Odpoczynek. Re...,0


In [34]:
X_1_COLUMN = 'nlp_2_4_5'
X_2_COLUMN = 'age'
X_3_COLUMN = 'sex'
Y_COLUMN = 'GD_category'

In [46]:
from sklearn.model_selection import train_test_split

cleaned_df = working_data[~df[X_1_COLUMN].isna()]
working_df = cleaned_df.filter([X_2_COLUMN, X_1_COLUMN, X_3_COLUMN, Y_COLUMN])
df_train, df_test = train_test_split(working_df, test_size = 0.2, random_state=23)

X_train = df_train.drop(Y_COLUMN, axis=1)
Y_train = df_train[Y_COLUMN].values

X_test = df_test.drop(Y_COLUMN, axis=1)
Y_test = df_test[Y_COLUMN].values

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15672\957479674.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cleaned_df = working_data[~df[X_1_COLUMN].isna()]


161

In [72]:
tokenized_train_x = bert_transformer.transform(X_train[X_1_COLUMN])
reshaped_x_2 = [[item] for item in X_train[X_2_COLUMN]]
reshaped_x_3 = [item for item in X_train[X_3_COLUMN]]

all_train_x = np.hstack((tokenized_train_x, reshaped_x_2, reshaped_x_3))

In [73]:
tokenized_test_x = bert_transformer.transform(X_test[X_1_COLUMN])
reshaped_x_2_test = [[item] for item in X_test[X_2_COLUMN]]
reshaped_x_3_test = [item for item in X_test[X_3_COLUMN]]

all_test_x = np.hstack((tokenized_test_x, reshaped_x_2_test, reshaped_x_3_test))

In [3]:
from sklearn import svm
vector_classification = svm.SVC(kernel='poly', C=0.1, degree=3, coef0=-100)
vector_classification.fit(all_train_x, Y_train)

results = vector_classification.predict(all_test_x)
results

NameError: name 'all_train_x' is not defined

In [ ]:
hit = 0
missed = 0
for prediction, true_data in zip(results, Y_test):
    if true_data == prediction:
        hit += 1
    else:
        missed += 1
 
hit, missed

(28, 13)